# Download data 


Downloading the ECG 2020 datasets and copy to google drive

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/ptb-xl/1.0.1/ 

In [ ]:
# %cp -r -np /content/physionet.org/files/ptb-xl /content/gdrive/My\ Drive/ECG2020new

In [ ]:
# #%cd /content/physionet.org/files/ptb-xl/
# %cd 20000/

In [ ]:
# %ls

In [ ]:
# %pwd

In [ ]:
#%cd physionet.org/files/ptb-xl/1.0.1/

Load GPU

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Link to Google Drive Import library

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Install library

In [ ]:
!pip install wfdb

     |████████████████████████████████| 122kB 15.4MB/s 


Import library

In [ ]:
import numpy as np
import pandas as pd
import wfdb
import matplotlib.pyplot as plt
import ast
import os
import tensorflow as tf
from keras.utils import to_categorical

import tensorflow_datasets as tfds

from sklearn.utils import resample
from sklearn.model_selection import train_test_split


# Get the label from csv, set up the basic raw data, get the features

In [ ]:
#get path
path='/content/gdrive/My Drive/ECG2020/1.0.1/'
# path = '/content/gdrive/.shortcut-targets-by-id/1ca8px2VUzY6uBCgo13kYoW0_JuynXfG7/1.0.1/'
tf_save_path = '/content/gdrive/My Drive/tfrecord/'

In [ ]:
def get_label_info(path):
  def add_path(str2):
      return path + str2
  def aggregate_diagnostic(y_dic):
      tmp = []
      for key in y_dic.keys():
          if key in agg_df.index:
              tmp.append(agg_df.loc[key].diagnostic_class)
      if len(tmp) == 0:
        return "NL"
      return list(set(tmp))[0]
  Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
  Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

      # Load scp_statements.csv for diagnostic aggregation
  agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
  agg_df = agg_df[agg_df.diagnostic == 1]

      # Apply diagnostic superclass
      #target label of the classification
  Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
  Y['new_filename'] = Y.filename_hr.map(add_path)
  return Y[["new_filename","diagnostic_superclass","age","sex"]]

In [ ]:
y = get_label_info(path)
y

,new_filename,diagnostic_superclass,age,sex
ecg_id,,,,
1,/content/gdrive/My Drive/ECG2020/1.0.1/records...,NORM,56.0,1
2,/content/gdrive/My Drive/ECG2020/1.0.1/records...,NORM,19.0,0
3,/content/gdrive/My Drive/ECG2020/1.0.1/records...,NORM,37.0,1
4,/content/gdrive/My Drive/ECG2020/1.0.1/records...,NORM,24.0,0
5,/content/gdrive/My Drive/ECG2020/1.0.1/records...,NORM,19.0,1
...,...,...,...,...
21833,/content/gdrive/My Drive/ECG2020/1.0.1/records...,STTC,67.0,1
21834,/content/gdrive/My Drive/ECG2020/1.0.1/records...,NORM,93.0,0
21835,/content/gdrive/My Drive/ECG2020/1.0.1/records...,STTC,59.0,1


In [ ]:
y = y[y.diagnostic_superclass !="DELETE"]
y.diagnostic_superclass.value_counts()

NORM    9490
STTC    5250
MI      4042
CD      1709
HYP      939
NL       407
Name: diagnostic_superclass, dtype: int64

In [ ]:
y = y.dropna(how='any',axis=0) 
y.diagnostic_superclass.value_counts()

NORM    9484
STTC    5211
MI      4020
CD      1695
HYP      935
NL       403
Name: diagnostic_superclass, dtype: int64

Convert the label into numercial data

In [ ]:
numeric_label = {"diagnostic_superclass":{"NORM":0,"CD":1,"MI":2,"STTC":3,"HYP":4,"NL":5}}

In [ ]:
y = y.replace(numeric_label)
y

,new_filename,diagnostic_superclass,age,sex
ecg_id,,,,
1,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,56.0,1
2,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,19.0,0
3,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,37.0,1
4,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,24.0,0
5,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,19.0,1
...,...,...,...,...
21833,/content/gdrive/My Drive/ECG2020/1.0.1/records...,3,67.0,1
21834,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,93.0,0
21835,/content/gdrive/My Drive/ECG2020/1.0.1/records...,3,59.0,1


Resampling the data label

In [75]:
#up sampling the miniority class of train dataset

df_0 = y[y.diagnostic_superclass==0]
df_1 = y[y.diagnostic_superclass==1]
df_2 = y[y.diagnostic_superclass==2]
df_3 = y[y.diagnostic_superclass==3]
df_4 = y[y.diagnostic_superclass==4]
df_0 = resample(df_0, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_1 = resample(df_1, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_2 = resample(df_2, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_3 = resample(df_3, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results
df_4 = resample(df_4, 
                                 replace=True,     # sample with replacement
                                 n_samples=5000,    # to match majority class
                                 random_state=123) # reproducible results                                
df_ = pd.concat([df_0,df_1,df_2,df_3,df_4],ignore_index=True)
df_.diagnostic_superclass.value_counts()

4    5000
3    5000
2    5000
1    5000
0    5000
Name: diagnostic_superclass, dtype: int64

In [76]:
#shuffle the dataframe and re_index the dataset
def shuffle_dataset(df):
  df = df.sample(frac=1)
  df = df.reset_index()
  return df
shuffle_dataset(df_)

,index,new_filename,diagnostic_superclass,age,sex
0,22064,/content/gdrive/My Drive/ECG2020/1.0.1/records...,4,46.0,0
1,20656,/content/gdrive/My Drive/ECG2020/1.0.1/records...,4,39.0,0
2,4172,/content/gdrive/My Drive/ECG2020/1.0.1/records...,0,75.0,0
3,19107,/content/gdrive/My Drive/ECG2020/1.0.1/records...,3,40.0,1
4,22160,/content/gdrive/My Drive/ECG2020/1.0.1/records...,4,80.0,1
...,...,...,...,...,...
24995,7471,/content/gdrive/My Drive/ECG2020/1.0.1/records...,1,70.0,1
24996,23970,/content/gdrive/My Drive/ECG2020/1.0.1/records...,4,82.0,1
24997,17305,/content/gdrive/My Drive/ECG2020/1.0.1/records...,3,72.0,1
24998,19586,/content/gdrive/My Drive/ECG2020/1.0.1/records...,3,71.0,1


In [77]:
#split the train test dataset
train_df, test_df = train_test_split(df_, test_size = 0.2)
#split the train validation dataset
train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [78]:
print("train dataset information:")
print(train_df.info())
print()
print("test dataset information:")
print(test_df.info())
print()
print("validation dataset information:")
print(val_df.info())

train dataset information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 16000 entries, 4981 to 2740
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   new_filename           16000 non-null  object 
 1   diagnostic_superclass  16000 non-null  int64  
 2   age                    16000 non-null  float64
 3   sex                    16000 non-null  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 625.0+ KB
None

test dataset information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 14057 to 7501
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   new_filename           5000 non-null   object 
 1   diagnostic_superclass  5000 non-null   int64  
 2   age                    5000 non-null   float64
 3   sex                    5000 non-null   int64  
dtypes: float64(1), int64(2), obj

# Read data into Tensorflow data pipeline

TensorFlow Encode and Decode base Function

In [94]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def save_tfrecords(data, label, sex, age, desfile):
    with tf.io.TFRecordWriter(desfile) as writer:
      features = tf.train.Features(
                feature = {
                    "data": _bytes_feature(tf.io.serialize_tensor(data)),
                    "label":_int64_feature(label),
                    "sex":_int64_feature(sex),
                    "age":_float_feature(age)
                }
            )
      example = tf.train.Example(features = features)
      serialized = example.SerializeToString()
      writer.write(serialized)

**here can load the raw data directly to memory, sampling_rate 100 means the 100hz dataset, physinet provide 100hz dataset for user easier to use but we use 500 hz dataset in our project here**

In [ ]:
# here can load the raw data directly to memory, sampling_rate 100 means the 100hz dataset, physinet provide 100hz dataset for user easier to use
# but we use 500 hz dataset in our project here

# def load_raw_data(path):
#     print("start")
#     sampling_rate = 100
#     def aggregate_diagnostic(y_dic):
#       tmp = []
#       for key in y_dic.keys():
#           if key in agg_df.index:
#               tmp.append(agg_df.loc[key].diagnostic_class)
#       return list(set(tmp))
#     # load and convert annotation data
#     Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
#     Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

#     # Load scp_statements.csv for diagnostic aggregation
#     agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
#     agg_df = agg_df[agg_df.diagnostic == 1]

#     # Apply diagnostic superclass
#     #target label of the classification
#     Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
#     print(len(Y.diagnostic_superclass))
#     #Y.reset_index()
#     # 100 HZ dataset
#     # if sampling_rate == 100:
#     #     data = [wfdb.rdrecord(path+f).p_signal.T for f in Y.filename_lr]
#     #     #print(data)
#     # else:
#     # 500 HZ array dataset
#     for i in range(1,len(Y)+1):
#       f = Y.filename_hr[i]
#       # print("file name:")
#       # print(f)
#       sex = Y.sex[i]
#       # print("sex:")
#       # print(sex)
#       age = Y.age[i]
#       # print("age:")
#       # print(age)
#       data =wfdb.rdrecord(path+f).p_signal.T
#       # print("data:"+str(i))
#       # print(data)
#       tf_path = os.path.join(tf_save_path,f+'.tfrecords')
#       # print("tf_path:")
#       # print(tf_path)
#       label = Y.diagnostic_superclass[i]
#       # print("label:")
#       # print(label)
#       save_tfrecords(data,label,sex,age,tf_path)
#     #data = [wfdb.rdrecord(path+f).p_signal.T for f in Y.filename_hr]
#     #data = np.array([signal for signal, meta in data])
#     print('finish')
#     return data, Y.diagnostic_superclass

In [95]:
#tensor flow function
def load_data(df,batch_size,i):
  #print(file_path)
  #df.reindex()
  end_index = (i+1)*batch_size
  if end_index > df.index[-1]:
    end_index = df.index[-1]
  start_index = i*batch_size
  path_list = df['new_filename'][start_index:end_index]
  #print(i)
  #age_tensor = tf.constant(df['age'][start_index: end_index],dtype="int8")
  #sex_tensor =tf.constant(df['sex'][start_index: end_index],dtype="int8")
  #label_one_hot = tf.one_hot(df['diagnostic_superclass'][start_index: end_index],5)
  #label_tensor = tf.constant(label_one_hot,dtype="float32")
  #file = file_path.numpy().decode('UTF-8')
  #print(file)
  data_list = []
  for j in range(len(path_list)):
    #print(len(path_list))
    #print(path_list.index[j])
    file_path = path_list[path_list.index[j]]
    #print(file_path)
    data = wfdb.rdrecord(file_path).p_signal.T.astype("float16")
    data_list.append(data)
  data_array = np.array(data_list)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array,age_tensor,sex_tensor,label_tensor))
  #set tensorflow feature
  label_tensor = tf.constant(train_df['diagnostic_superclass'][start_index: end_index],5)
  dataset = tf.data.Dataset.from_tensor_slices((data_array, label_tensor))
  return dataset

def one_hot_encode(signal,label):
    if label == 0:
      label = np.array([1.0,.0,.0,.0,.0]).astype("float16")
    elif label == 1:
      label = np.array([.0,1.0,.0,.0,.0]).astype("float16")
    elif label ==3:
      label = np.array([.0,.0,1.0,.0,.0]).astype("float16")
    elif label ==4:
      label = np.array([.0,.0,.0,1.0,.0]).astype("float16")
    else: 
      label = np.array([.0,.0,.0,.0,1.0]).astype("float16")
    return signal , label

# data preprocessing 

In [96]:
import pywt

def denoise(sig_data):
    # wavedec
    coeffs = pywt.wavedec(data=sig_data, wavelet='db8', level=9)
    cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
    #print('here')
    # threshold
    threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
    cD1.fill(0)
    cD2.fill(0)
    for i in range(1, len(coeffs) - 2):
       coeffs[i] = pywt.threshold(coeffs[i], threshold)

    rdata = pywt.waverec(coeffs=coeffs, wavelet='db8')
    #print(rdata)
    return rdata

def denoise_for_tf_data(data,label):
  # Generating numpy array 
  data_array = np.array(data)
  #print(data_array)
  for i, sig_array in enumerate(data_array):     
    #print(type(sig_array))
    # plt.figure()
    # plt.plot(sig_array[0:1024]) 
    # plt.show()
    #print(type(sig_array))
    sig_length = len(sig_array)
    sig_array = denoise(sig_array)
    #print(sig_array)
    #plt.plot(sig_array[0:1024])
    #plt.show()
    data_array[i] = sig_array[0:sig_length]
  #print(data_array.shape)
  data = tf.convert_to_tensor(data_array)
  #print(data.numpy())
  return data,label

In [83]:
def get_dataset(df,batch_size, i):
    dataset = load_data(df, batch_size,i)
    ds_one_hot = dataset.map(one_hot_encode)
    ds_denoise = ds_one_hot.map(lambda x,y: 
                              tuple(tf.py_function(denoise_for_tf_data,[x,y],[tf.float64,tf.float16])))
    ds_batch = ds_one_hot.batch(32)
    return ds_batch

In [84]:
def show(data, label):
  for sig_data in data:
    plt.figure()
    plt.title(label)
    plt.plot(sig_data[0:1024])
    plt.show()

In [97]:
# test function
ds_test0 = get_dataset(train_df,100, 0)
def show_dataset(dataset):
  iterator = iter(dataset)
  raw_example = next(iterator)
  parsed_data = raw_example[0].numpy()
  label = raw_example[1].numpy()
  print('batch_data:',parsed_data, 'shape:', parsed_data.shape, '\n batcah_labels:',label, 'shape:', label.shape)

show_dataset(ds_test0)

batch_data: [[[ 0.06   0.06   0.06  ... -0.19  -0.19  -0.19 ]
  [ 0.05   0.05   0.05  ... -0.155 -0.155 -0.155]
  [-0.01  -0.01  -0.01  ...  0.035  0.035  0.035]
  ...
  [-0.355 -0.355 -0.355 ...  0.135  0.135  0.135]
  [-2.574 -2.574 -2.574 ...  0.     0.     0.   ]
  [-0.22  -0.22  -0.22  ...  0.2    0.2    0.2  ]]

 [[-0.085 -0.085 -0.085 ...  0.015  0.015  0.015]
  [-0.055 -0.055 -0.055 ...  0.     0.     0.   ]
  [ 0.03   0.03   0.03  ... -0.015 -0.015 -0.015]
  ...
  [-0.285 -0.285 -0.285 ... -0.135 -0.135 -0.135]
  [-0.185 -0.185 -0.185 ... -0.085 -0.085 -0.085]
  [-0.115 -0.115 -0.115 ... -0.065 -0.065 -0.065]]

 [[-0.075 -0.075 -0.075 ...  0.015  0.015  0.015]
  [-0.085 -0.085 -0.085 ...  0.085  0.085  0.085]
  [-0.01  -0.01  -0.01  ...  0.07   0.07   0.07 ]
  ...
  [-0.145 -0.145 -0.145 ...  0.285  0.285  0.285]
  [ 0.14   0.14   0.14  ...  0.03   0.03   0.03 ]
  [-0.315 -0.315 -0.315 ...  0.15   0.15   0.15 ]]

 ...

 [[-0.025 -0.025 -0.025 ... -0.12  -0.12  -0.12 ]
  [-0.20

# Modeling

In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [86]:
def buildModel():

    newModel = tf.keras.models.Sequential([
                                           
        tf.keras.layers.InputLayer(input_shape=(12,5000)),

        tf.keras.layers.Conv1D(filters=4, kernel_size=21, strides=1, padding='SAME', activation='relu'),
       
        tf.keras.layers.AvgPool1D(pool_size=1, strides=4, padding='SAME'),
        
        tf.keras.layers.Conv1D(filters=16, kernel_size=23, strides=1, padding='SAME', activation='relu'),
        
        tf.keras.layers.AvgPool1D(pool_size=1, strides=4, padding='SAME'),
       
        tf.keras.layers.Conv1D(filters=32, kernel_size=25, strides=1, padding='SAME', activation='elu'),
       
        tf.keras.layers.AvgPool1D(pool_size=1, strides=8, padding='SAME'),
       
        tf.keras.layers.Conv1D(filters=64, kernel_size=27, strides=1, padding='SAME', activation='relu'),
       
        tf.keras.layers.Flatten(),
       
        tf.keras.layers.Dense(128, activation='relu'),
       
        tf.keras.layers.Dropout(rate=0.2),
       
        tf.keras.layers.Dense(5, activation='softmax')
    ])

    # newModel.compile(optimizer = 'adam' , 
    #               loss = "categorical_crossentropy", metrics=["accuracy"])

    return newModel

In [98]:
# with tpu_strategy.scope():
model_test = buildModel()
model_test.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 12, 4)             420004    
_________________________________________________________________
average_pooling1d_9 (Average (None, 3, 4)              0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 3, 16)             1488      
_________________________________________________________________
average_pooling1d_10 (Averag (None, 1, 16)             0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 1, 32)             12832     
_________________________________________________________________
average_pooling1d_11 (Averag (None, 1, 32)             0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 1, 64)            

In [99]:
import datetime
from keras.optimizers import RMSprop, Adam
  #tbcb = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  write_graph=True, write_images=True)
%load_ext tensorboard

log_dir = "logs/fit_Feb26/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)
model_test.compile(optimizer = 'adam', 
                  loss = "categorical_crossentropy", metrics=["accuracy"])

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Training

In [100]:
def train_model(train_df,batch_size , epochs):
  for i in range(int(len(train_df)/batch_size)+1):
    print('round:', i)  
    ds_batch = get_dataset(train_df,batch_size,i)
    train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
    train_cache = train_prefetch.cache()
    model_test.fit(train_cache, epochs=epochs,callbacks = [tensorboard_callback])

In [101]:
train_df.index.value_counts()

6141     1
14978    1
4759     1
6806     1
661      1
        ..
23809    1
21760    1
1274     1
7417     1
2049     1
Length: 16000, dtype: int64

In [102]:
train_model(train_df, 512, 1)

round: 0
16/16 [==============================] - 1s 13ms/step - loss: 1.6082 - accuracy: 0.2401
round: 1
16/16 [==============================] - 0s 11ms/step - loss: 1.6062 - accuracy: 0.2227
round: 2
16/16 [==============================] - 0s 10ms/step - loss: 1.6094 - accuracy: 0.1895
round: 3
16/16 [==============================] - 0s 10ms/step - loss: 1.5944 - accuracy: 0.2344
round: 4
16/16 [==============================] - 0s 11ms/step - loss: 1.5907 - accuracy: 0.2344
round: 5
6/6 [==============================] - 0s 28ms/step - loss: 1.5950 - accuracy: 0.2111
round: 6


ValueError: ignored